This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

# Best practices for the real world

## Getting the most out of your models

### Hyperparameter optimization

#### Using KerasTuner

In [1]:
# !pip install keras-tuner -q

**A KerasTuner model-building function**

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

**A KerasTuner `HyperModel`**

In [3]:
import kerastuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

C:\Users\mccar\AppData\Local\Temp\ipykernel_1828\1594418805.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [4]:
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=100,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

In [5]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [8]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=5,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 100 Complete [00h 00m 32s]
val_accuracy: 0.9658499956130981

Best val_accuracy So Far: 0.9739499986171722
Total elapsed time: 01h 00m 31s
INFO:tensorflow:Oracle triggered exit


**Querying the best hyperparameter configurations**

In [9]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

In [10]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [12]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
391/391 [==============================] - 5s 9ms/step - loss: 0.4280 - accuracy: 0.8864 - val_loss: 0.2270 - val_accuracy: 0.9365
Epoch 2/100
391/391 [==============================] - 4s 9ms/step - loss: 0.2162 - accuracy: 0.9388 - val_loss: 0.1789 - val_accuracy: 0.9499
Epoch 3/100
391/391 [==============================] - 3s 7ms/step - loss: 0.1650 - accuracy: 0.9530 - val_loss: 0.1491 - val_accuracy: 0.9593
Epoch 4/100
391/391 [==============================] - 2s 6ms/step - loss: 0.1356 - accuracy: 0.9613 - val_loss: 0.1334 - val_accuracy: 0.9632
Epoch 5/100
391/391 [==============================] - 3s 7ms/step - loss: 0.1162 - accuracy: 0.9667 - val_loss: 0.1235 - val_accuracy: 0.9648
Epoch 6/100
391/391 [==============================] - 3s 8ms/step - loss: 0.1018 - accuracy: 0.9707 - val_loss: 0.1175 - val_accuracy: 0.9668
Epoch 7/100
391/391 [==============================] - 3s 7ms/step - loss: 0.0912 - accuracy: 0.9740 - val_loss: 0.1147 - val_accuracy: 0.9677

In [13]:
best_models = tuner.get_best_models(top_n)

#### The art of crafting the right search space

#### The future of hyperparameter tuning: automated machine learning

### Model ensembling

## Scaling-up model training

### Speeding up training on GPU with mixed precision

#### Understanding floating-point precision

In [14]:
import tensorflow as tf
import numpy as np
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array)
tf_tensor.dtype

tf.float64

In [15]:
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array, dtype="float32")
tf_tensor.dtype

tf.float32

#### Mixed-precision training in practice

In [16]:
from tensorflow import keras
keras.mixed_precision.set_global_policy("mixed_float16")

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


### Multi-GPU training

#### Getting your hands on two or more GPUs

#### Single-host, multi-device synchronous training

### TPU training

#### Using a TPU via Google Colab

#### Leveraging step fusing to improve TPU utilization

## Summary